In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
data = pd.read_csv("../data/preprocessed_data.csv")
data.info()
def cluster_data(df, features=None, k_range=range(2, 11), max_samples=10000, random_state=42, verbose=True):
    
    
    
    # Sample Of Data
    sample_size = min(len(df), max_samples)
    samples = df.sample(n=sample_size, random_state=random_state)  
    
    # Search for best K
    scores = []
    labels_dict = {}
    
    # LOOP
    for k in k_range:
    
        kmedoids = KMedoids(n_clusters=k, random_state=random_state, metric='euclidean')
        labels = kmedoids.fit_predict(samples)
        labels_dict[k] = labels
        medoids_dict = {k: kmedoids}
        
        #silhouette score
        score = silhouette_score(samples, labels)
        scores.append(score)
    
    # Find optimal k (skip k=1 if it's in the range)
    best_k = k_range[scores.index(max(scores))]

    df_clustered = df.copy()

    # Add Cluster Column to Data
    df_clustered.loc[samples.index, 'Cluster'] = labels_dict[best_k] 
    
    # Get the best model
    best_kmedoids = KMedoids(n_clusters=best_k, random_state=random_state).fit(samples)
    
    if len(df_clustered) > sample_size:

        # Assign remaining points to nearest medoid center
        remaining_indices = df_clustered.index.difference(samples.index)
        remaining_data = df.loc[remaining_indices]

        # Predict clusters for remaining points
        remaining_labels = best_kmedoids.predict(remaining_data)
        df_clustered.loc[remaining_indices, 'Cluster'] = remaining_labels
    
    return df_clustered, best_k, scores
def analyze_clusters(df, cluster_col='Cluster'):

    features = df.select_dtypes(include='number').columns.drop(cluster_col)

    print(df[cluster_col].value_counts().sort_index())
    return df.groupby(cluster_col)[features].mean()
X = data.drop(columns=['Category'])  # Numerical Data
Y = data['Category']

df_clustered, best_k, scores = cluster_data(X)  


# Analyze cluster characteristics
cluster_means = analyze_clusters(df_clustered)
print(cluster_means)

df_clustered.info()
df_clustered["Category"]=data["Category"]
df_clustered.columns
df_clustered.to_csv("../data/Kmedoid_data.csv", index=False)

ModuleNotFoundError: No module named 'sklearn_extra'